# Avaliação e Previsão de Agentes e Fenômenos que Mais Causam Impactos Socioeconômicos na Região Sudeste do Brasil

# **0.** Instalação das Dependências
<a id="instalar"></a>

In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# **1.** Importação das Bibliotecas
<a id="bibliotecas"></a>

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# **2.** Importação das Bases de Dados
<a id="bases"></a>

In [5]:
rend_trabalho = pd.read_csv('../data/raw/Indic_Geo_Rend.csv', comment=None , header=1)
ind_trabalho = pd.read_csv('../data/raw/Indic_Geo.csv', comment=None , header=1)
gini = pd.read_csv('../data/raw/Indice_Gini_Geo.csv', comment=None , header=1)
nivel_instrucao = pd.read_csv('../data/raw/Instr_Geo.csv', comment=None , header=1)
populacao = pd.read_csv('../data/raw/Pop_Geo.csv', comment=None , header=1)
saneamento = pd.read_csv('../data/raw/Sane_Geo_pes.csv', comment=None , header=1)
analfabetos = pd.read_csv('../data/raw/TaxaAnalf_Geo.csv', comment=None , header=1)
mortalidade = pd.read_csv('../data/raw/TaxaMort_Geo.csv', comment=None , header=1)
plano = pd.read_csv('../data/raw/Benef_Plano_Geo.csv', comment=None , header=1)
leitos = pd.read_csv('../data/raw/Leit_IntTot_Geo.csv', comment=None , header=1)
ocupacao = pd.read_csv('../data/raw/CondOcup_Geo.csv', comment=None , header=1)
idh = pd.read_csv('../data/raw/Ind_Desen_Hum.csv', comment=None , header=1)
pib = pd.read_csv('../data/raw/Prod_Int_Bruto.csv', comment=None , header=1)

## Visualização das Bases de Dados

In [6]:
display(rend_trabalho)

,Estado,Ano,Rendimento médio real habitual do trabalho principal(R$/mês),Rendimento médio real habitual de todos os trabalhos(R$/mês),Rendimento-hora médio real habitual do trabalho principal(R$/hora),Rendimento-hora médio real habitual de todos os trabalhos(R$/hora)\t
0,Rondônia,2023,2636,2695,18.0,18.0
1,Acre,2023,2338,2358,15.0,15.0
2,Amazonas,2023,2308,2367,14.4,14.4
3,Roraima,2023,2701,2791,16.7,16.6
4,Pará,2023,2176,2265,13.8,13.8
...,...,...,...,...,...,...
130,Rio Grande do Sul,2019,3110,3211,18.9,18.9
131,Mato Grosso do Sul,2019,2910,3011,18.2,18.2
132,Mato Grosso,2019,2781,2846,17.4,17.2
133,Goiás,2019,2530,2597,15.2,15.2


In [7]:
display(ind_trabalho)

,Estado,Ano,População em idade de trabalhar,População na força de trabalho,População ocupada,População ocupada em trabalhos formais,População desocupada,População na força de trabalho potencial,População subutilizada,Taxa de participação,Nível de ocupação,Taxa de formalização,Taxa de desocupação,Taxa composta de subutilização
0,Rondônia,2023,1496,851,824,486,27,19.00,56.0,56.9,55.1,59.0,3.2,6.5
1,Acre,2023,691,342,316,152,25,36.00,68.0,49.4,45.7,48.0,7.5,18.0
2,Amazonas,2023,3182,1946,1752,703,194,139.00,424.0,61.2,55.1,40.1,9.9,20.3
3,Roraima,2023,449,280,262,120,18,13.38,62.5,58.4,45.7,6.6,12.8,NaN
4,Pará,2023,6862,4194,3869,1449,325,476.00,1123.0,61.1,56.4,37.4,7.7,24.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Rio Grande do Sul,2019,9396,6280,5788,4045,492,219.00,1011.0,66.8,61.6,69.9,7.8,15.6
131,Mato Grosso do Sul,2019,2113,1460,1345,778,116,88.00,276.0,69.1,63.6,57.8,7.9,17.8
132,Mato Grosso,2019,2649,1845,1694,1006,151,85.00,309.0,69.6,63.9,59.4,8.2,16.0
133,Goiás,2019,5643,3817,3409,1933,408,179.00,731.0,67.6,60.4,56.7,10.7,18.3


In [8]:
display(gini)

,Estado,Ano,Índice Gini
0,Rondônia,2023,0.455
1,Acre,2023,0.511
2,Amazonas,2023,0.512
3,Roraima,2023,0.520
4,Pará,2023,0.501
...,...,...,...
131,Rio Grande do Sul,2019,0.482
132,Mato Grosso do Sul,2019,0.483
133,Mato Grosso,2019,0.454
134,Goiás,2019,0.460


In [9]:
display(nivel_instrucao)

,Estado,Ano,Total Pessoas Ocupadas(1 000 pessoas),Sem instrução ou fundamental incompleto,Ensino fundamental completo ou médio incompleto,Ensino médio completo ou superior incompleto,Ensino superior completo
0,Rondônia,2023,824,27.3,15.7,38.9,18.1
1,Acre,2023,316,22.7,14.5,39.8,23.0
2,Amazonas,2023,1752,21.4,11.2,47.5,19.9
3,Roraima,2023,262,17.2,13.7,45.3,23.8
4,Pará,2023,3869,27.5,15.0,40.9,16.7
...,...,...,...,...,...,...,...
130,Rio Grande do Sul,2019,5788,23.6,17.4,39.6,19.4
131,Mato Grosso do Sul,2019,1345,28.2,16.8,34.9,20.1
132,Mato Grosso,2019,1694,26.7,18.3,37.4,17.5
133,Goiás,2019,3409,25.8,16.0,39.8,18.4


In [10]:
display(nivel_instrucao)

,Estado,Ano,Total Pessoas Ocupadas(1 000 pessoas),Sem instrução ou fundamental incompleto,Ensino fundamental completo ou médio incompleto,Ensino médio completo ou superior incompleto,Ensino superior completo
0,Rondônia,2023,824,27.3,15.7,38.9,18.1
1,Acre,2023,316,22.7,14.5,39.8,23.0
2,Amazonas,2023,1752,21.4,11.2,47.5,19.9
3,Roraima,2023,262,17.2,13.7,45.3,23.8
4,Pará,2023,3869,27.5,15.0,40.9,16.7
...,...,...,...,...,...,...,...
130,Rio Grande do Sul,2019,5788,23.6,17.4,39.6,19.4
131,Mato Grosso do Sul,2019,1345,28.2,16.8,34.9,20.1
132,Mato Grosso,2019,1694,26.7,18.3,37.4,17.5
133,Goiás,2019,3409,25.8,16.0,39.8,18.4


In [11]:
display(populacao)

,Estado,Ano,População
0,Acre,2019,865
1,Alagoas,2019,3330
2,Amazonas,2019,3983
3,Amapá,2019,838
4,Bahia,2019,14854
...,...,...,...
130,Rio Grande do Sul,2023,11206
131,Santa Catarina,2023,7913
132,Sergipe,2023,2282
133,São Paulo,2023,45846


In [12]:
display(saneamento)

,Estado,Ano,Saneamento Basico Total(1 000 pessoas),Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)
0,Rondônia,2019,1841.0,22.7
1,Acre,2023,914.0,37.8
2,Amazonas,2023,4221.0,43.6
3,Roraima,2023,610.0,54.6
4,Pará,2023,8902.0,18.4
...,...,...,...,...
130,Rio Grande do Sul,2019,11359.0,68.4
131,Mato Grosso do Sul,2019,2709.0,49.9
132,Mato Grosso,2019,3431.0,36.9
133,Goiás,2019,7021.0,59.6


In [13]:
display(analfabetos)

,Estado,Ano,Taxa de Analfabetismo
0,Rondônia,2023,5.1
1,Acre,2023,9.4
2,Amazonas,2023,5.1
3,Roraima,2023,4.2
4,Pará,2023,6.9
...,...,...,...
130,Rio Grande do Sul,2019,2.4
131,Mato Grosso do Sul,2019,4.6
132,Mato Grosso,2019,5.6
133,Goiás,2019,4.6


In [14]:
display(mortalidade)

,Estado,Ano,Taxa Total Mortalidade,0 a 14 anos,15 a 29 anos,30 a 44 anos,45 a 59 anos,60 anos ou mais,60 a 69 anos,70 anos ou mais
0,Rondônia,2023,5.73,1.10,1.48,2.31,5.45,29.98,14.45,53.54
1,Acre,2023,4.78,1.54,1.32,2.10,5.11,30.23,13.93,52.75
2,Amazonas,2023,4.80,1.53,1.60,2.42,5.34,29.47,14.13,52.66
3,Roraima,2023,4.74,2.52,1.57,2.46,5.75,28.21,14.66,49.97
4,Pará,2023,5.08,1.33,1.44,2.26,4.94,28.61,13.43,49.27
...,...,...,...,...,...,...,...,...,...,...
130,Rio Grande do Sul,2019,7.97,0.94,1.13,1.95,5.66,33.60,15.17,56.90
131,Mato Grosso do Sul,2019,6.06,1.08,1.22,2.07,5.81,32.07,14.83,55.31
132,Mato Grosso,2019,5.16,1.26,1.37,2.15,5.47,29.16,14.62,51.07
133,Goiás,2019,5.88,1.16,1.46,2.14,5.33,30.99,14.38,53.79


In [15]:
display(plano)

,Estado,T1 2019,T2 2019,T3 2019,T4 2019,T1 2020,T2 2020,T3 2020,T4 2020,T1 2021,...,T3 2021,T4 2021,T1 2022,T2 2022,T3 2022,T4 2022,T1 2023,T2 2023,T3 2023,T4 2023
0,Rondônia,99941,104567,104202,103809,105345,107436,106437,112245,109397,...,109470,109143,110414,111001,112095,112299,113119,115597,116071,120689
1,Acre,15112,14374,14409,15464,15573,15018,15544,15774,15552,...,16825,16934,17300,17219,17282,17279,17062,17349,18595,19973
2,Amazonas,400005,409042,415795,430642,439485,421017,439357,457801,459628,...,462127,490663,497536,505700,515848,509477,542588,549525,553702,558178
3,Roraima,9213,9476,9812,10665,10803,10289,10805,11289,10825,...,10972,10730,10902,11118,11045,11491,11372,11631,11768,11691
4,Pará,410263,417289,414379,426214,428410,418499,440902,457614,460211,...,505659,506517,499187,514505,508455,509878,510563,518613,525875,538871
5,Amapá,43889,43420,44229,44497,44445,44021,45834,48269,48598,...,49644,49510,48457,48235,50965,52097,51416,50313,47846,48172
6,Tocantins,49181,120511,121949,126251,126445,111537,62062,63728,63182,...,130005,130278,129395,119472,120848,121521,122139,121469,124778,127249
7,Maranhão,197051,194141,195059,201519,206131,199345,214092,223894,224574,...,240966,245027,252267,265950,272590,281604,287596,293209,314189,324883
8,Piauí,84202,84390,85628,91800,96288,89132,93756,105918,108355,...,120771,128841,130013,138822,145142,151008,157527,169434,179582,191980
9,Ceará,951107,964397,944408,966518,992173,958343,979538,996734,1022691,...,1020264,1048814,1051472,1088977,1126009,1157246,1174417,1194993,1222830,1321364


In [16]:
display(leitos)

,Estado,Ano,Número mensal médio de leitos de internação (total)
0,Rondônia,2019,4477
1,Acre,2019,1469
2,Amazonas,2019,5701
3,Roraima,2019,1071
4,Pará,2019,14412
...,...,...,...
130,Rio Grande do Sul,2023,29615
131,Mato Grosso do Sul,2023,5907
132,Mato Grosso,2023,7533
133,Goiás,2023,18728


In [17]:
display(ocupacao)

,Estado,Ano,Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas),Domicílio Próprio - já pago,Domicílio Próprio - pagando,Domicílio Alugado,Domicílio Cedido por empregador,Domicílio Cedido por familiar,Domicílio Cedido de outra forma,Outra Forma
0,Rondônia,2023,1841.0,68.8,1.5,19.9,2.0,6.2,1.5,0.1
1,Acre,2023,914.0,79.8,0.7,10.2,2.0,6.4,0.6,0.3
2,Amazonas,2023,4221.0,73.8,1.9,15.5,0.7,7.2,0.7,0.1
3,Roraima,2023,610.0,59.0,2.8,26.1,2.7,7.7,1.7,0.1
4,Pará,2023,8902.0,74.1,3.0,13.5,1.4,6.8,0.8,0.3
...,...,...,...,...,...,...,...,...,...,...
131,Rio Grande do Sul,2019,11359.0,68.6,8.9,15.2,1.0,5.2,0.8,0.4
132,Mato Grosso do Sul,2019,2709.0,51.9,11.5,21.8,5.9,7.8,0.8,0.2
133,Mato Grosso,2019,3431.0,60.0,6.1,23.0,5.2,4.7,0.9,0.3
134,Goiás,2019,7021.0,54.5,11.1,23.1,3.1,7.2,0.7,0.4


In [18]:
display(idh)

,Estado,Ano,IDH
0,Acre,2019,0.739
1,Alagoas,2019,0.687
2,Amazonas,2019,0.726
3,Amapá,2019,0.737
4,Bahia,2019,0.718
...,...,...,...
130,Rio Grande do Sul,2023,NaN
131,Santa Catarina,2023,NaN
132,Sergipe,2023,NaN
133,São Paulo,2023,NaN


In [19]:
display(pib)

,Estado,Ano,PIB
0,Acre,2019,15630017.0
1,Alagoas,2019,58963729.0
2,Amazonas,2019,108181091.0
3,Amapá,2019,17496661.0
4,Bahia,2019,293240504.0
...,...,...,...
125,Rio Grande do Sul,2023,NaN
126,Santa Catarina,2023,NaN
127,Sergipe,2023,NaN
128,São Paulo,2023,NaN


# **3.** Análise dos Dados antes do Tratamento
<a id="analise1"></a>

## Verificação do Tamanho das Colunas e Linhas das Bases de Dados

In [20]:
rend_trabalho.shape

(135, 6)

In [21]:
ind_trabalho.shape

(135, 14)

In [22]:
gini.shape

(136, 3)

In [23]:
nivel_instrucao.shape

(135, 7)

In [24]:
populacao.shape

(135, 3)

In [25]:
saneamento.shape

(135, 4)

In [26]:
analfabetos.shape

(135, 3)

In [27]:
mortalidade.shape

(135, 10)

In [28]:
plano.shape

(27, 21)

In [29]:
leitos.shape

(135, 3)

In [30]:
ocupacao.shape

(136, 10)

In [31]:
idh.shape

(135, 3)

In [32]:
pib.shape

(130, 3)

Análise das Bases de Dados: Tamanho das colunas e linhas. Informações de todos os estados do Brasil. Tabela de plano possui um volume maior de informações do que as outras

## Verificação de Valores Nulos nas Bases de Dados

In [33]:
rend_trabalho.isnull().sum()

Estado                                                                  0
Ano                                                                     0
Rendimento médio real habitual do trabalho principal(R$/mês)            0
Rendimento médio real habitual de todos os trabalhos(R$/mês)            0
Rendimento-hora médio real habitual do trabalho principal(R$/hora)      0
Rendimento-hora médio real habitual de todos os trabalhos(R$/hora)\t    0
dtype: int64

In [34]:
ind_trabalho.isnull().sum()

Estado                                       0
Ano                                          0
População em idade de trabalhar              0
População na força de trabalho               0
População ocupada                            0
População ocupada em trabalhos formais       0
População desocupada                         0
População na força de trabalho potencial     0
População subutilizada                       0
Taxa de participação                         0
Nível de ocupação                            0
Taxa de formalização                         0
Taxa de desocupação                          0
Taxa composta de subutilização              29
dtype: int64

In [38]:
(ind_trabalho.isnull().sum() / len(ind_trabalho)) * 100

Estado                                       0.000000
Ano                                          0.000000
População em idade de trabalhar              0.000000
População na força de trabalho               0.000000
População ocupada                            0.000000
População ocupada em trabalhos formais       0.000000
População desocupada                         0.000000
População na força de trabalho potencial     0.000000
População subutilizada                       0.000000
Taxa de participação                         0.000000
Nível de ocupação                            0.000000
Taxa de formalização                         0.000000
Taxa de desocupação                          0.000000
Taxa composta de subutilização              21.481481
dtype: float64

In [35]:
gini.isnull().sum()

Estado         0
Ano            0
Índice Gini    0
dtype: int64

In [36]:
nivel_instrucao.isnull().sum()

Estado                                             0
Ano                                                0
Total Pessoas Ocupadas(1 000 pessoas)              0
Sem instrução ou fundamental incompleto            0
Ensino fundamental completo ou médio incompleto    0
Ensino médio completo ou superior incompleto       0
Ensino superior completo                           0
dtype: int64

In [37]:
populacao.isnull().sum()

Estado       0
Ano          0
População    0
dtype: int64

In [39]:
saneamento.isnull().sum()

Estado                                                                                   0
Ano                                                                                      0
Saneamento Basico Total(1 000 pessoas)                                                  54
Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)    54
dtype: int64

In [42]:
(saneamento.isnull().sum() / len(saneamento)) * 100

Estado                                                                                   0.0
Ano                                                                                      0.0
Saneamento Basico Total(1 000 pessoas)                                                  40.0
Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)    40.0
dtype: float64

In [40]:
analfabetos.isnull().sum()

Estado                    0
Ano                       0
Taxa de Analfabetismo    54
dtype: int64

In [41]:
(analfabetos.isnull().sum() / len(analfabetos)) * 100

Estado                    0.0
Ano                       0.0
Taxa de Analfabetismo    40.0
dtype: float64

In [43]:
mortalidade.isnull().sum()

Estado                    0
Ano                       0
Taxa Total Mortalidade    0
0 a 14 anos               0
15 a 29 anos              0
30 a 44 anos              0
45 a 59 anos              0
60 anos ou mais           0
60 a 69 anos              0
70 anos ou mais           0
dtype: int64

In [44]:
plano.isnull().sum()

Estado     0
T1 2019    0
T2 2019    0
T3 2019    0
T4 2019    0
T1 2020    0
T2 2020    0
T3 2020    0
T4 2020    0
T1 2021    0
T2 2021    0
T3 2021    0
T4 2021    0
T1 2022    0
T2 2022    0
T3 2022    0
T4 2022    0
T1 2023    0
T2 2023    0
T3 2023    0
T4 2023    0
dtype: int64

In [45]:
leitos.isnull().sum()

Estado                                                 0
Ano                                                    0
Número mensal médio de leitos de internação (total)    0
dtype: int64

In [46]:
ocupacao.isnull().sum()

Estado                                                                0
Ano                                                                   0
Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)    54
Domicílio Próprio - já pago                                          54
Domicílio Próprio - pagando                                          54
Domicílio Alugado                                                    54
Domicílio Cedido por empregador                                      54
Domicílio Cedido por familiar                                        54
Domicílio Cedido de outra forma                                      54
Outra Forma                                                          57
dtype: int64

In [47]:
(ocupacao.isnull().sum() / len(ocupacao)) * 100

Estado                                                                0.000000
Ano                                                                   0.000000
Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)    39.705882
Domicílio Próprio - já pago                                          39.705882
Domicílio Próprio - pagando                                          39.705882
Domicílio Alugado                                                    39.705882
Domicílio Cedido por empregador                                      39.705882
Domicílio Cedido por familiar                                        39.705882
Domicílio Cedido de outra forma                                      39.705882
Outra Forma                                                          41.911765
dtype: float64

In [48]:
idh.isnull().sum()

Estado     0
Ano        0
IDH       54
dtype: int64

In [50]:
(idh.isnull().sum() / len(idh)) * 100

Estado     0.0
Ano        0.0
IDH       40.0
dtype: float64

In [49]:
pib.isnull().sum()

Estado     0
Ano        0
PIB       26
dtype: int64

In [51]:
(pib.isnull().sum() / len(pib)) * 100

Estado     0.0
Ano        0.0
PIB       20.0
dtype: float64

Análise dos Dados: 
ind trabalho, 21% dados faltantes na coluna
saneamento, 40% dados faltantes em cada uma das 2 colunas
analfabetos, 40% dados faltantes em cada uma das 2 colunas
ocupação, aproximadamente 40% dados faltantes em cada uma das 8 colunas
idh, 40% dados faltantes na coluna
pib, 20% dados faltantes na coluna


## Visualização dos Tipos de Dados em Cada Base

In [52]:
rend_trabalho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Estado                                                               135 non-null    object 
 1   Ano                                                                  135 non-null    int64  
 2   Rendimento médio real habitual do trabalho principal(R$/mês)         135 non-null    int64  
 3   Rendimento médio real habitual de todos os trabalhos(R$/mês)         135 non-null    int64  
 4   Rendimento-hora médio real habitual do trabalho principal(R$/hora)   135 non-null    float64
 5   Rendimento-hora médio real habitual de todos os trabalhos(R$/hora)	  135 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 6.5+ KB


In [53]:
ind_trabalho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 14 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Estado                                    135 non-null    object 
 1   Ano                                       135 non-null    int64  
 2   População em idade de trabalhar           135 non-null    int64  
 3   População na força de trabalho            135 non-null    int64  
 4   População ocupada                         135 non-null    int64  
 5   População ocupada em trabalhos formais    135 non-null    int64  
 6   População desocupada                      135 non-null    int64  
 7   População na força de trabalho potencial  135 non-null    float64
 8   População subutilizada                    135 non-null    float64
 9   Taxa de participação                      135 non-null    float64
 10  Nível de ocupação                     

In [54]:
gini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Estado       136 non-null    object 
 1   Ano          136 non-null    int64  
 2   Índice Gini  136 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.3+ KB


In [55]:
nivel_instrucao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Estado                                           135 non-null    object 
 1   Ano                                              135 non-null    int64  
 2   Total Pessoas Ocupadas(1 000 pessoas)            135 non-null    int64  
 3   Sem instrução ou fundamental incompleto          135 non-null    float64
 4   Ensino fundamental completo ou médio incompleto  135 non-null    float64
 5   Ensino médio completo ou superior incompleto     135 non-null    float64
 6   Ensino superior completo                         135 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 7.5+ KB


In [56]:
populacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Estado     135 non-null    object
 1   Ano        135 non-null    int64 
 2   População  135 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ KB


In [57]:
saneamento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 4 columns):
 #   Column                                                                                Non-Null Count  Dtype  
---  ------                                                                                --------------  -----  
 0   Estado                                                                                135 non-null    object 
 1   Ano                                                                                   135 non-null    int64  
 2   Saneamento Basico Total(1 000 pessoas)                                                81 non-null     float64
 3   Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)  81 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 4.3+ KB


In [58]:
analfabetos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Estado                 135 non-null    object 
 1   Ano                    135 non-null    int64  
 2   Taxa de Analfabetismo  81 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.3+ KB


In [59]:
mortalidade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Estado                  135 non-null    object 
 1   Ano                     135 non-null    object 
 2   Taxa Total Mortalidade  135 non-null    float64
 3   0 a 14 anos             135 non-null    float64
 4   15 a 29 anos            135 non-null    float64
 5   30 a 44 anos            135 non-null    float64
 6   45 a 59 anos            135 non-null    float64
 7   60 anos ou mais         135 non-null    float64
 8   60 a 69 anos            135 non-null    float64
 9   70 anos ou mais         135 non-null    float64
dtypes: float64(8), object(2)
memory usage: 10.7+ KB


In [60]:
plano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Estado   27 non-null     object
 1   T1 2019  27 non-null     int64 
 2   T2 2019  27 non-null     int64 
 3   T3 2019  27 non-null     int64 
 4   T4 2019  27 non-null     int64 
 5   T1 2020  27 non-null     int64 
 6   T2 2020  27 non-null     int64 
 7   T3 2020  27 non-null     int64 
 8   T4 2020  27 non-null     int64 
 9   T1 2021  27 non-null     int64 
 10  T2 2021  27 non-null     int64 
 11  T3 2021  27 non-null     int64 
 12  T4 2021  27 non-null     int64 
 13  T1 2022  27 non-null     int64 
 14  T2 2022  27 non-null     int64 
 15  T3 2022  27 non-null     int64 
 16  T4 2022  27 non-null     int64 
 17  T1 2023  27 non-null     int64 
 18  T2 2023  27 non-null     int64 
 19  T3 2023  27 non-null     int64 
 20  T4 2023  27 non-null     int64 
dtypes: int64(20), object(1)
memory usage: 4.6

In [61]:
leitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Estado                                               135 non-null    object
 1   Ano                                                  135 non-null    int64 
 2   Número mensal médio de leitos de internação (total)  135 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ KB


In [62]:
ocupacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 10 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Estado                                                             136 non-null    object 
 1   Ano                                                                136 non-null    int64  
 2   Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)  82 non-null     float64
 3   Domicílio Próprio - já pago                                        82 non-null     float64
 4   Domicílio Próprio - pagando                                        82 non-null     float64
 5   Domicílio Alugado                                                  82 non-null     float64
 6   Domicílio Cedido por empregador                                    82 non-null     float64
 7   Domicílio Cedido por famil

In [63]:
idh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Estado  135 non-null    object 
 1   Ano     135 non-null    int64  
 2   IDH     81 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.3+ KB


In [64]:
pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Estado  130 non-null    object 
 1   Ano     130 non-null    int64  
 2   PIB     104 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ KB


# **4.** Tratamento dos Dados
<a id="tratamento"></a>

## Filtração dos Dados para Apenas a Região Sudeste do Brasil

In [65]:
rend_trabalho = rend_trabalho[(rend_trabalho['Estado'] == 'Minas Gerais') | (rend_trabalho['Estado'] == 'Espírito Santo') | (rend_trabalho['Estado'] == 'Rio de Janeiro') | (rend_trabalho['Estado'] == 'São Paulo')]
ind_trabalho = ind_trabalho[(ind_trabalho['Estado'] == 'Minas Gerais') | (ind_trabalho['Estado'] == 'Espírito Santo') | (ind_trabalho['Estado'] == 'Rio de Janeiro') | (ind_trabalho['Estado'] == 'São Paulo')]
gini = gini[(gini['Estado'] == 'Minas Gerais') | (gini['Estado'] == 'Espírito Santo') | (gini['Estado'] == 'Rio de Janeiro') | (gini['Estado'] == 'São Paulo')]
nivel_instrucao = nivel_instrucao[(nivel_instrucao['Estado'] == 'Minas Gerais') | (nivel_instrucao['Estado'] == 'Espírito Santo') | (nivel_instrucao['Estado'] == 'Rio de Janeiro') | (nivel_instrucao['Estado'] == 'São Paulo')]
populacao = populacao[(populacao['Estado'] == 'Minas Gerais') | (populacao['Estado'] == 'Espírito Santo') | (populacao['Estado'] == 'Rio de Janeiro') | (populacao['Estado'] == 'São Paulo')]
saneamento = saneamento[(saneamento['Estado'] == 'Minas Gerais') | (saneamento['Estado'] == 'Espírito Santo') | (saneamento['Estado'] == 'Rio de Janeiro') | (saneamento['Estado'] == 'São Paulo')]
analfabetos = analfabetos[(analfabetos['Estado'] == 'Minas Gerais') | (analfabetos['Estado'] == 'Espírito Santo') | (analfabetos['Estado'] == 'Rio de Janeiro') | (analfabetos['Estado'] == 'São Paulo')]
mortalidade = mortalidade[(mortalidade['Estado'] == 'Minas Gerais') | (mortalidade['Estado'] == 'Espírito Santo') | (mortalidade['Estado'] == 'Rio de Janeiro') | (mortalidade['Estado'] == 'São Paulo')]
leitos = leitos[(leitos['Estado'] == 'Minas Gerais') | (leitos['Estado'] == 'Espírito Santo') | (leitos['Estado'] == 'Rio de Janeiro') | (leitos['Estado'] == 'São Paulo')]
plano = plano[(plano['Estado'] == 'Minas Gerais') | (plano['Estado'] == 'Espírito Santo') | (plano['Estado'] == 'Rio de Janeiro') | (plano['Estado'] == 'São Paulo')]
ocupacao = ocupacao[(ocupacao['Estado'] == 'Minas Gerais') | (ocupacao['Estado'] == 'Espírito Santo') | (ocupacao['Estado'] == 'Rio de Janeiro') | (ocupacao['Estado'] == 'São Paulo')]
idh = idh[(idh['Estado'] == 'Minas Gerais') | (idh['Estado'] == 'Espírito Santo') | (idh['Estado'] == 'Rio de Janeiro') | (idh['Estado'] == 'São Paulo')]
pib = pib[(pib['Estado'] == 'Minas Gerais') | (pib['Estado'] == 'Espírito Santo') | (pib['Estado'] == 'Rio de Janeiro') | (pib['Estado'] == 'São Paulo')]

## Substituição dos Valores Nulos das Colunas que os possuem, por suas Respectivas Médias, de seus Respectivos Estados

### Coluna "Taxa composta de subutilização" da Base `ind_trabalho`

In [66]:
# Coluna "Taxa composta de subutilização" da Base 'ind_trabalho'
media_mg = ind_trabalho.loc[ind_trabalho['Estado'] == 'Minas Gerais','Taxa composta de subutilização'].mean()
ind_trabalho.loc[ind_trabalho['Estado'] == 'Minas Gerais', 'Taxa composta de subutilização'] = ind_trabalho.loc[
ind_trabalho['Estado'] == 'Minas Gerais', 'Taxa composta de subutilização'].fillna(media_mg)

media_es = ind_trabalho.loc[ind_trabalho['Estado'] == 'Espírito Santo','Taxa composta de subutilização'].mean()
ind_trabalho.loc[ind_trabalho['Estado'] == 'Espírito Santo', 'Taxa composta de subutilização'] = ind_trabalho.loc[
ind_trabalho['Estado'] == 'Espírito Santo', 'Taxa composta de subutilização'].fillna(media_es)

media_rj = ind_trabalho.loc[ind_trabalho['Estado'] == 'Rio de Janeiro','Taxa composta de subutilização'].mean()
ind_trabalho.loc[ind_trabalho['Estado'] == 'Rio de Janeiro', 'Taxa composta de subutilização'] = ind_trabalho.loc[
ind_trabalho['Estado'] == 'Rio de Janeiro', 'Taxa composta de subutilização'].fillna(media_rj)

media_sp = ind_trabalho.loc[ind_trabalho['Estado'] == 'São Paulo','Taxa composta de subutilização'].mean()
ind_trabalho.loc[ind_trabalho['Estado'] == 'São Paulo', 'Taxa composta de subutilização'] = ind_trabalho.loc[
ind_trabalho['Estado'] == 'São Paulo', 'Taxa composta de subutilização'].fillna(media_sp)

In [67]:
ind_trabalho.isnull().sum()

Estado                                      0
Ano                                         0
População em idade de trabalhar             0
População na força de trabalho              0
População ocupada                           0
População ocupada em trabalhos formais      0
População desocupada                        0
População na força de trabalho potencial    0
População subutilizada                      0
Taxa de participação                        0
Nível de ocupação                           0
Taxa de formalização                        0
Taxa de desocupação                         0
Taxa composta de subutilização              0
dtype: int64

### Coluna "Saneamento Basico Total(1 000 pessoas)" da Base `saneamento`

In [68]:
media_mg = saneamento.loc[saneamento['Estado'] == 'Minas Gerais','Saneamento Basico Total(1 000 pessoas)'].mean()
saneamento.loc[saneamento['Estado'] == 'Minas Gerais', 'Saneamento Basico Total(1 000 pessoas)'] = saneamento.loc[
saneamento['Estado'] == 'Minas Gerais', 'Saneamento Basico Total(1 000 pessoas)'].fillna(media_mg)

media_es = saneamento.loc[saneamento['Estado'] == 'Espírito Santo','Saneamento Basico Total(1 000 pessoas)'].mean()
saneamento.loc[saneamento['Estado'] == 'Espírito Santo', 'Saneamento Basico Total(1 000 pessoas)'] = saneamento.loc[
saneamento['Estado'] == 'Espírito Santo', 'Saneamento Basico Total(1 000 pessoas)'].fillna(media_es)

media_rj = saneamento.loc[saneamento['Estado'] == 'Rio de Janeiro','Saneamento Basico Total(1 000 pessoas)'].mean()
saneamento.loc[saneamento['Estado'] == 'Rio de Janeiro', 'Saneamento Basico Total(1 000 pessoas)'] = saneamento.loc[
saneamento['Estado'] == 'Rio de Janeiro', 'Saneamento Basico Total(1 000 pessoas)'].fillna(media_rj)

media_sp = saneamento.loc[saneamento['Estado'] == 'São Paulo','Saneamento Basico Total(1 000 pessoas)'].mean()
saneamento.loc[saneamento['Estado'] == 'São Paulo', 'Saneamento Basico Total(1 000 pessoas)'] = saneamento.loc[
saneamento['Estado'] == 'São Paulo', 'Saneamento Basico Total(1 000 pessoas)'].fillna(media_sp)

### Coluna "Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)" da Base `saneamento`

In [69]:
media_mg = saneamento.loc[saneamento['Estado'] == 'Minas Gerais','Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].mean()
saneamento.loc[saneamento['Estado'] == 'Minas Gerais', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'] = saneamento.loc[
saneamento['Estado'] == 'Minas Gerais', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].fillna(media_mg)

media_es = saneamento.loc[saneamento['Estado'] == 'Espírito Santo','Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].mean()
saneamento.loc[saneamento['Estado'] == 'Espírito Santo', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'] = saneamento.loc[
saneamento['Estado'] == 'Espírito Santo', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].fillna(media_es)

media_rj = saneamento.loc[saneamento['Estado'] == 'Rio de Janeiro','Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].mean()
saneamento.loc[saneamento['Estado'] == 'Rio de Janeiro', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'] = saneamento.loc[
saneamento['Estado'] == 'Rio de Janeiro', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].fillna(media_rj)

media_sp = saneamento.loc[saneamento['Estado'] == 'São Paulo','Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].mean()
saneamento.loc[saneamento['Estado'] == 'São Paulo', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'] = saneamento.loc[
saneamento['Estado'] == 'São Paulo', 'Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)'].fillna(media_sp)

In [71]:
saneamento.isnull().sum()

Estado                                                                                  0
Ano                                                                                     0
Saneamento Basico Total(1 000 pessoas)                                                  0
Proporção de pessoas com acesso simultâneo aos três serviços de saneamento básico(%)    0
dtype: int64

### Coluna "Taxa de Analfabetismo" da Base `analfabetos`



In [72]:
media_mg = analfabetos.loc[analfabetos['Estado'] == 'Minas Gerais','Taxa de Analfabetismo'].mean()
analfabetos.loc[analfabetos['Estado'] == 'Minas Gerais', 'Taxa de Analfabetismo'] = analfabetos.loc[
analfabetos['Estado'] == 'Minas Gerais', 'Taxa de Analfabetismo'].fillna(media_mg)

media_es = analfabetos.loc[analfabetos['Estado'] == 'Espírito Santo','Taxa de Analfabetismo'].mean()
analfabetos.loc[analfabetos['Estado'] == 'Espírito Santo', 'Taxa de Analfabetismo'] = analfabetos.loc[
analfabetos['Estado'] == 'Espírito Santo', 'Taxa de Analfabetismo'].fillna(media_es)

media_rj = analfabetos.loc[analfabetos['Estado'] == 'Rio de Janeiro','Taxa de Analfabetismo'].mean()
analfabetos.loc[analfabetos['Estado'] == 'Rio de Janeiro', 'Taxa de Analfabetismo'] = analfabetos.loc[
analfabetos['Estado'] == 'Rio de Janeiro', 'Taxa de Analfabetismo'].fillna(media_rj)

media_sp = analfabetos.loc[analfabetos['Estado'] == 'São Paulo','Taxa de Analfabetismo'].mean()
analfabetos.loc[analfabetos['Estado'] == 'São Paulo', 'Taxa de Analfabetismo'] = analfabetos.loc[
analfabetos['Estado'] == 'São Paulo', 'Taxa de Analfabetismo'].fillna(media_sp)

In [73]:
analfabetos.isnull().sum()

Estado                   0
Ano                      0
Taxa de Analfabetismo    0
dtype: int64

### Coluna "Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)" da Base `ocupacao`

In [74]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)'].fillna(media_sp)

### Coluna "Domicílio Próprio - já pago" da Base `ocupacao`

In [75]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Próprio - já pago'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Próprio - já pago'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Próprio - já pago'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Próprio - já pago'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Próprio - já pago'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Próprio - já pago'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Próprio - já pago'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Próprio - já pago'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Próprio - já pago'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Próprio - já pago'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Próprio - já pago'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Próprio - já pago'].fillna(media_sp)

### Coluna "Domicílio Próprio - pagando" da Base `ocupacao`

In [76]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Próprio - pagando'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Próprio - pagando'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Próprio - pagando'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Próprio - pagando'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Próprio - pagando'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Próprio - pagando'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Próprio - pagando'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Próprio - pagando'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Próprio - pagando'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Próprio - pagando'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Próprio - pagando'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Próprio - pagando'].fillna(media_sp)

### Coluna "Domicílio Alugado" da Base `ocupacao`

In [77]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Alugado'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Alugado'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Alugado'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Alugado'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Alugado'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Alugado'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Alugado'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Alugado'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Alugado'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Alugado'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Alugado'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Alugado'].fillna(media_sp)

### Coluna "Domicílio Cedido por empregador" da Base `ocupacao`

In [78]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Cedido por empregador'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido por empregador'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido por empregador'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Cedido por empregador'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido por empregador'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido por empregador'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Cedido por empregador'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido por empregador'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido por empregador'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Cedido por empregador'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido por empregador'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido por empregador'].fillna(media_sp)

### Coluna "Domicílio Cedido por familiar" da Base `ocupacao`

In [79]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Cedido por familiar'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido por familiar'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido por familiar'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Cedido por familiar'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido por familiar'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido por familiar'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Cedido por familiar'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido por familiar'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido por familiar'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Cedido por familiar'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido por familiar'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido por familiar'].fillna(media_sp)

### Coluna "Domicílio Cedido de outra forma" da Base `ocupacao`

In [80]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Domicílio Cedido de outra forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido de outra forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Domicílio Cedido de outra forma'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Domicílio Cedido de outra forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido de outra forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Domicílio Cedido de outra forma'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Domicílio Cedido de outra forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido de outra forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Domicílio Cedido de outra forma'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Domicílio Cedido de outra forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido de outra forma'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Domicílio Cedido de outra forma'].fillna(media_sp)

### Coluna "Outra Forma" da Base `ocupacao`

In [81]:
media_mg = ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais','Outra Forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Minas Gerais', 'Outra Forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Minas Gerais', 'Outra Forma'].fillna(media_mg)

media_es = ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo','Outra Forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Espírito Santo', 'Outra Forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Espírito Santo', 'Outra Forma'].fillna(media_es)

media_rj = ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro','Outra Forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'Rio de Janeiro', 'Outra Forma'] = ocupacao.loc[
ocupacao['Estado'] == 'Rio de Janeiro', 'Outra Forma'].fillna(media_rj)

media_sp = ocupacao.loc[ocupacao['Estado'] == 'São Paulo','Outra Forma'].mean()
ocupacao.loc[ocupacao['Estado'] == 'São Paulo', 'Outra Forma'] = ocupacao.loc[
ocupacao['Estado'] == 'São Paulo', 'Outra Forma'].fillna(media_sp)

In [82]:
ocupacao.isnull().sum()

Estado                                                               0
Ano                                                                  0
Total Pessoas por Condição de Ocupação a domicílio(1 000 pessoas)    0
Domicílio Próprio - já pago                                          0
Domicílio Próprio - pagando                                          0
Domicílio Alugado                                                    0
Domicílio Cedido por empregador                                      0
Domicílio Cedido por familiar                                        0
Domicílio Cedido de outra forma                                      0
Outra Forma                                                          0
dtype: int64

## Padronização da Base `plano` para Dados Anuais, Utilizando a Média Anual de cada Estado

In [83]:
plano_reformado = plano.melt(id_vars=["Estado"], var_name="Trimestre", value_name="Número de beneficiários de plano de saúde")
plano_reformado["Ano"] = plano_reformado["Trimestre"].str.split().str[1].astype(int)
plano_anual = plano_reformado.groupby(["Estado", "Ano"], as_index=False)["Número de beneficiários de plano de saúde"].mean()
plano = plano_anual.copy()
display(plano)


,Estado,Ano,Número de beneficiários de plano de saúde
0,Espírito Santo,2019,508849.75
1,Espírito Santo,2020,527951.25
2,Espírito Santo,2021,591777.00
3,Espírito Santo,2022,639377.75
4,Espírito Santo,2023,686278.50
5,Minas Gerais,2019,2070470.25
6,Minas Gerais,2020,2125927.75
7,Minas Gerais,2021,2329022.50
8,Minas Gerais,2022,2548886.25
9,Minas Gerais,2023,2701387.25


## Unificação das Bases de Dados

In [ ]:
bases =[rend_trabalho, ind_trabalho, gini, nivel_instrucao, populacao, saneamento, analfabetos, mortalidade, leitos, plano, ocupacao, idh, pib]
from functools import reduce

for df in bases:
    df.columns = df.columns.str.lower().str.strip()

for df in bases:
    df.loc[:, 'ano'] = pd.to_numeric(df['ano'], errors='coerce').astype('Int64')

database = reduce(lambda left, right: pd.merge(
    left, right, on=['estado', 'ano'], how='outer'), bases)
display(database)

### Importação da Base Unificada

database.to_csv('database.csv', index=False)
database.to_excel('database.xlsx', index=False)

# **5.** Análise Exploratória dos Dados
<a id="analise2"></a>

## Verificação das Informações da Base

### Quantidade de Linhas e Colunas

In [ ]:
database.shape

### Quantidade de Valores Nulos por Coluna

In [ ]:
database.isnull().sum()

### Informações dos Tipos dos Dados

In [ ]:
database.info()

## Verificando Infomações Estatísticas da Base

In [ ]:
database.describe()

Análise de Dados: A descrição estatística dos dados fornece um resumo conciso das principais características de cada coluna:

*   **count:** Indica a quantidade de valores não nulos em cada coluna. Ou seja, mostra quantos registros de fato contêm dados válidos.

*   **mean:** Representa a média aritmética dos dados de cada coluna, ou seja, a soma de todos os valores dividida pela quantidade de valores.

*   **std** (Desvio Padrão): Mede o grau de dispersão dos valores em relação à média. Um desvio padrão alto indica que os dados estão muito espalhados. Já um std baixo indica que os dados estão concentrados próximos da média.

*   **min**: Apresenta o menor valor encontrado na coluna.

*  **25%** (Primeiro Quartil - Q1): Indica que 25% dos dados estão abaixo desse valor. Representa o limite inferior da faixa de valores típicos.

*   **50%** (Mediana ou Segundo Quartil - Q2): O valor central da distribuição, indica que 50% dos dados estão abaixo dele.

*   **75%** (Terceiro Quartil - Q3): Indica que 75% dos dados estão abaixo desse valor. Representa o limite superior da faixa de valores "típicos".

*   **max:** Apresenta o maior valor encontrado na coluna.

---



De modo geral, a inspeção inicial dos dados revela que todas as colunas possuem valores válidos, sem ocorrência de dados nulos. Quanto à dispersão, o desvio padrão (STD) demonstra uma variabilidade significativa entre as variáveis, com algumas colunas apresentando alto grau de dispersão em relação à média e outras com baixa dispersão, o que em determinados casos pode ser favorável ou não á análise.


---


## Gerando Matrizes de Correlação dos Dados

### Matriz de Todos os Dados da Base 

In [ ]:
base_numeric = database.select_dtypes(include=['number'])
base_numeric.corr()

### Visualização da Matriz de Correlação por HeatMap

In [ ]:
base_numeric = database.select_dtypes(include=['number'])
sns.heatmap(base_numeric.corr())

## Análise de Matrizes Específicas da Base